Configuração inicial
--
Começo importando a biblioteca [Python Jira](https://jira.readthedocs.io/), a qual me permite fazer uso da API REST do Jira.

Em seguida crio um objeto JIRA, passando a autenticação básica (``username`` e ``password``) e o servidor onde está o Jira Software.

In [182]:
from jira import JIRA
jira = JIRA(basic_auth=('xb218744', 'RFbAHDs3dcJ2e2z'), options={'server': 'http://localhost:8080'})

Explorando o objeto ``jira``
--
Aqui, verifico os tipos de issues que estão configurados.

In [184]:
issue_types = jira.issue_types()
issue_types

[<JIRA IssueType: name='Tarefa', id='10003'>,
 <JIRA IssueType: name='Sub-tarefa', id='10004'>,
 <JIRA IssueType: name='História', id='10002'>,
 <JIRA IssueType: name='Problema', id='10005'>,
 <JIRA IssueType: name='Épico', id='10001'>,
 <JIRA IssueType: name='Sub-task', id='10000'>]

E aqui, exploro os status que podem ser atribuídos às issues.

In [185]:
statuses_types = jira.statuses()
statuses_types

[<JIRA Status: name='In Progress', id='3'>,
 <JIRA Status: name='To Do', id='10000'>,
 <JIRA Status: name='Done', id='10001'>]

Buscando os recursos necessários
--
Aqui, utilizo JQL (acrônimo para Jira Query Language) para definir alguns filtros para a busca por issues.

Defino o projeto como sendo ``PPJ``, além de que o tipo da issue deva estar entre os tipos padrões de issues (o que não inclui sub-task) e, por fim, que o tipo da issue não seja ``Épico`` (no caso utilizo o id 10001, para garantir que não haja problema com o idioma).

Após definir a variável ``jql``, crio uma variável chamada ``issues`` a qual armazena o resultado da função ``jira.search_issues()``, onde foi definida com os parâmetros ``maxResults=100``, para a quantidade máxima de resultados e o parâmetro ``expand='changelog'``, que expande a busca além das issues, retornando também os changelogs de cada uma.

Por último, imprimo a lista de issues retornadas.

In [190]:
jql = 'project=PPJ AND issuetype in standardIssueTypes() AND issueType != 10001'
issues = jira.search_issues(jql, maxResults=100, expand='changelog')
issues

[<JIRA Issue: key='PPJ-13', id='10012'>,
 <JIRA Issue: key='PPJ-12', id='10011'>,
 <JIRA Issue: key='PPJ-11', id='10010'>,
 <JIRA Issue: key='PPJ-10', id='10009'>,
 <JIRA Issue: key='PPJ-9', id='10008'>,
 <JIRA Issue: key='PPJ-8', id='10007'>,
 <JIRA Issue: key='PPJ-7', id='10006'>,
 <JIRA Issue: key='PPJ-6', id='10005'>,
 <JIRA Issue: key='PPJ-5', id='10004'>,
 <JIRA Issue: key='PPJ-4', id='10003'>,
 <JIRA Issue: key='PPJ-3', id='10002'>,
 <JIRA Issue: key='PPJ-2', id='10001'>,
 <JIRA Issue: key='PPJ-1', id='10000'>]

In [ ]:
#df.to_csv('test.csv', index=False)